In [1]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import ShuffleSplit, KFold
import matplotlib.pyplot as plt
from scipy import sparse
from itertools import product
from time import time
import seaborn
import pickle
from collections import Counter
import heapq

In [2]:
#load everything
num_ing = 8023
num_items = 8576
num_rec = 178265
recipe_decoder = pickle.load( open('data-cleaned/recipe_decoder.pkl', 'rb') )
ingr_decoder   = pickle.load( open('data-cleaned/ingredient_decoder.pkl', 'rb') )
tag_decoder    = pickle.load( open('data-cleaned/tag_decoder.pkl', 'rb') )
R     = sparse.load_npz("data-cleaned/recipes.npz")
Rhat  = sparse.load_npz("data-cleaned/recipes_tfidf.npz")
U     = sparse.load_npz("data-cleaned/user_train.npz")
Uhat  = sparse.load_npz("data-cleaned/user_train_tfidf.npz")
Utest = sparse.load_npz("data-cleaned/user_test.npz")
col_sum = np.load("data-cleaned/recipe_col_sum.npy")
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [3]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from recipeClasses import *
from userClasses import *

In [4]:
#prepare testing data and splitting data
user_test = Utest[:,-1].toarray().flatten().astype('int')
y = np.zeros((user_test.shape[0], 2), dtype='int')-1
for i in range(len(y)):
    recipes = Utest[i].nonzero()[1][:-1]
    if len(recipes) == 1:
        y[i,0] = recipes
    elif len(recipes) == 2:
        y[i,:] = recipes
    else:
        raise ValueError("Someone reviewed 3 recipes!")

Urecipe = user_to_recipe(U[user_test][:100], 4)
y = y[:100]

In [7]:
pipe = Pipeline([("dr", TruncatedSVD(n_components=10)),
                    ("rdr", recipeKNN())])

pipe.fit(R)
pipe.score(Urecipe, y)

12.086000000000002

In [10]:
pipe.predict(Urecipe)[1], y[1]

(array([ 53205, 107335,  79548, 177124, 157501]), array([177317,     -1]))

In [6]:
pipe = Pipeline([("dr", TruncatedSVD(n_components=10)),
                    ("rdr", userKNN())])

pipe.fit(U)
pipe.score(U[user_test][:100], y)

7.741999999999999